Source: (교재) 파이썬 머신러닝 완벽가이드

## 추천 시스템 타입


Demographic Filtering : 인구통계학적 특징에 기반

Content Based Filtering : 콘텐츠의 특성 기반 (장르 속성과 같이 사용자가 특정 영화를 감상하고 좋아했다면 비슷한 특성/속성의 다른 영화 추천)

Collaborative Filtering : A가 한가지 이슈에 관해서 B와 같은 의견을 갖는다면 다른 이슈에 대해서도 비슷한 의견을 가질 확률이 높을 것에 기반


In [36]:
#  데이터 로드
import pandas as pd
import numpy as np
import warnings;
warnings.filterwarnings('ignore')
movies = pd.read_csv('C:\\Users\\user\\Desktop\\소정\\프로그래밍\\Python\\Study\\tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1) # 영화 제목, 개요, 인기도, 평점, 투표 수 등 영화에 대한 다양한 메타 정보

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [37]:
movies_df=movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]


In [38]:
# 장르와 키워드의 경우 여러 개로 표시된 경우가 있으므로 전처리 필요 
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [39]:
from ast import literal_eval
# 문자열 -> 리스트 내부에 여러 딕셔너리로 구성된 객체로
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)
# -> 리스트 내 여러 개의 딕셔너리의 'name'에 해당하는 키를 찾아 리스트 객체로 변환
movies_df['genres']=movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


콘텐츠 기반 

1. 문자열로 변환된 genres칼럼을 count기반으로 피처 벡터화 변환

2. 피처 벡터화 행렬로 변환된 것으로 코사인 유사도 비교

3. 장르 유사도가 높은 영화중에 평점이 높은 순으로 영화 추천

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
movies_df['genres_literal']=movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


CountVectorizer : 토큰 리스트로 변환시켜 출현 빈도 벡터 행렬을 만들어줌

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [42]:
# 행별로 유사도 값이 높은 순으로 정렬된 위치 인덱스 값 추출
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])
print('0번 레코드의 경우 3494번 레코드가 가장 유사도가 높고, 그 다음이 813번 레코드')

[[   0 3494  813 ... 3038 3037 2401]]
0번 레코드의 경우 3494번 레코드가 가장 유사도가 높고, 그 다음이 813번 레코드


In [43]:
# 장르 유사도에 따라 영화를 추천하는 함수 생성
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie=df[df['title']==title_name]
    title_index=title_movie.index.values
    indexes=sorted_ind[title_index,:(top_n)]
    similar_indexes=indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [44]:
similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1
